In [1]:
import os
import rasterio
import numpy as np

def reclassify_raster(input_raster_path, output_raster_path, reclass_rules, output_nodata_val=-9999):
    """
    Reclassifies a raster based on a list of rules.

    Args:
        input_raster_path (str): Path to the input raster file (e.g., merged NDVI).
        output_raster_path (str): Path to save the new reclassified raster file.
        reclass_rules (list): A list of rules. Each rule is a sublist in the format:
                              [from_value, to_value, new_class_value].
                              Example: [[-1.0, 0.05, 1], [0.05, 0.2, 5], ...]
        output_nodata_val (int): The NoData value for the output raster.
    """
    if not os.path.exists(input_raster_path):
        print(f"!!! Error: Input file not found at '{input_raster_path}'.")
        return

    print(f"\n--- Starting Reclassification for: {os.path.basename(input_raster_path)} ---")

    try:
        with rasterio.open(input_raster_path) as src:
            # Read the source raster as a NumPy array
            data = src.read(1)
            # Copy the metadata from the source raster
            meta = src.meta.copy()
            
            # Get the source NoData value
            input_nodata_val = src.nodata
            if input_nodata_val is None:
                print("Warning: Input raster does not have a NoData value set.")
            
            # Create lists for conditions and choices for np.select
            conditions = []
            choices = []
            
            print("Applying reclassification rules:")
            for rule in reclass_rules:
                from_val, to_val, new_val = rule
                print(f"  - Values from {from_val} to {to_val} will be reclassified to {new_val}")
                # Create a condition for the current rule
                # Note: The upper bound (to_val) is inclusive.
                conditions.append((data > from_val) & (data <= to_val))
                choices.append(new_val)
            
            # --- Perform the reclassification using NumPy's select function ---
            # This is very efficient. It checks conditions in order and assigns the
            # corresponding choice. The 'default' value is assigned to anything
            # that doesn't meet any condition.
            reclassified_data = np.select(conditions, choices, default=output_nodata_val)

            # --- Handle the original NoData values ---
            # Ensure that pixels that were NoData in the input are also NoData in the output.
            if input_nodata_val is not None:
                reclassified_data[data == input_nodata_val] = output_nodata_val

            # --- Update metadata for the output file ---
            meta.update(
                dtype=rasterio.int16,  # Suitability scores are integers
                count=1,
                nodata=output_nodata_val,
                compress='lzw'
            )

            # --- Write the reclassified data to a new GeoTIFF file ---
            with rasterio.open(output_raster_path, 'w', **meta) as dst:
                dst.write(reclassified_data.astype(rasterio.int16), 1)
        
        print(f"\nSuccessfully reclassified raster and saved to: {output_raster_path}")

    except Exception as e:
        print(f"An error occurred during reclassification: {e}")

# --- MAIN SCRIPT EXECUTION ---
if __name__ == "__main__":
    # --- USER: DEFINE YOUR INPUTS AND RULES HERE ---

    # 1. Define the input merged NDVI file and the desired output path
    input_raster_path = "C:/Users/Lenovo/Documents/Dam_Suitability_Analysis/DATA/Processed_Rasters/NDVI_merged.tif"
    output_raster_path = "C:/Users/Lenovo/Documents/Dam_Suitability_Analysis/DATA//Processed_Rasters/Suitability_Layers/NDVI_Suitability.tif"
    
    # Create the output directory if it doesn't exist
    output_dir = os.path.dirname(output_raster_path)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # 2. Define the Reclassification Rules for NDVI
    # Each sublist is a rule: [from_value, to_value, new_suitability_score]
    # You should adjust the 'from' and 'to' values based on your NDVI histogram and visual analysis.
    # The suitability scores are based on the logic that less vegetation is better for dam construction.
    ndvi_reclass_rules = [
        # Rule 1: Water
        # Values from -1.0 up to and including 0.05 get a new value of 1.
        [-1.0, 0.05, 1],  # Class 1: Water (can be treated as low suitability or excluded later)
        
        # Rule 2: Bare Soil / Low Vegetation (Most Suitable)
        # Values greater than 0.05 up to and including 0.2 get a new value of 5.
        [0.05, 0.2, 5],   # Class 5: Most Suitable
        
        # Rule 3: Moderate Vegetation
        # Values greater than 0.2 up to and including 0.5 get a new value of 3.
        [0.2, 0.5, 3],    # Class 3: Moderately Suitable
        
        # Rule 4: High Vegetation (Least Suitable)
        # Values greater than 0.5 up to and including 1.0 get a new value of 1.
        [0.5, 1.0, 1]     # Class 1: Least Suitable
    ]

    # --- Run the Reclassification ---
    reclassify_raster(input_raster_path, output_raster_path, ndvi_reclass_rules)
    
    print("\nReclassification process finished.")



--- Starting Reclassification for: NDVI_merged.tif ---
Applying reclassification rules:
  - Values from -1.0 to 0.05 will be reclassified to 1
  - Values from 0.05 to 0.2 will be reclassified to 5
  - Values from 0.2 to 0.5 will be reclassified to 3
  - Values from 0.5 to 1.0 will be reclassified to 1

Successfully reclassified raster and saved to: C:/Users/Lenovo/Documents/Dam_Suitability_Analysis/DATA//Processed_Rasters/Suitability_Layers/NDVI_Suitability.tif

Reclassification process finished.
